In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os
# from scipy.special import kv
# from scipy.integrate import quad
from importlib import reload
import pickle
from scipy.stats import maxwell

from pulsars import Pulsars, c, e_charge, e_mass, h
import speiser_fun as sf 
import speiser_fun_cyl as sfc
import speiser_integ as si
import speiser_plots as sp
import speiser_model as sm
import speiser_spec as ss

In [2]:
k = 500.
name = 'crab'
pulsar = Pulsars(k)[name]

gamma1 = 1.
B_0 = pulsar['Blc']
omegaB = (e_charge*B_0/(e_mass*c))    #γυροσυχνότητα
Rlc = pulsar['rlc']*omegaB/c
Delta = pulsar['rlc']/k*omegaB/c
q = -1.

# gamma0 = np.array([1000., 1.])   #αρχικός παράγοντας Lorentz
# init_pos = np.array([0.5])

gamma0 = maxwell.rvs(loc = 50, scale = 800, size=4)
init_pos = np.array([0.25, 0.5])

delta = 50.    #πάχος του φύλλου ρεύματος, αδιάστατο, σε μονάδες [c/ωΒ]

T = 20.
N = 10**5
t = np.linspace(0.0, T*Delta, N)


In [3]:
#Δημιουργία λεξικού με τις παραμέτους της προσωμοίωσης σε αναγνώσιμη και δυαδική μορφή
parameters = {'pulsar_name':name, 'char magn field (Blc)':B_0, 'Radius of lc':Rlc, 'multiplicity':k, 'diss zone length':Delta,'diss zone width':delta, 
              'charge sign':q, 'runtime':T, 'number of t points':N, 'initial lorentz factors':gamma0, 
              'initial injection positions':init_pos}

dir_name = 'data'

#Δημιουργεία φακέλου για την αποθηκευση των δεδομένων
for i in range(1, 1000):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print('directory ', dir_name, 'created')
        break
    else:
        dir_name = 'data{}'.format(i)
        
with open('{}/parameters.txt'.format(dir_name), 'w') as par_file:
    for key, val in parameters.items():
        par_file.write('{} --> {}\n'.format(key, val))
with open('{}/par_dic.txt'.format(dir_name), 'wb') as par_file:
    pickle.dump(parameters, par_file)

directory  data8 created


In [4]:
for w in range(len(init_pos)):
    
    print(init_pos[w])
    
    (r, ur, phi, uphi, z_cyl, uz_cyl), dic_cyl = si.oloklirosi(gamma0, Rlc, Delta, delta, B_0, t, init_pos[w], q, T, 
                                                               sm.speiser_cyl, coord = 'cyl')
    
    with open('{}/cyl_{}_pos.npz'.format(dir_name, int(100*init_pos[w])), 'wb') as f:
        np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)
    

0.25
runtime for gamma0 = 2475 is 0h 6m 5s
runtime for gamma0 = 494 is 0h 2m 44s
runtime for gamma0 = 818 is 0h 4m 10s
total runtime = 0h 13m 0s
0.5
runtime for gamma0 = 2475 is 0h 8m 48s
runtime for gamma0 = 494 is 0h 3m 40s
runtime for gamma0 = 818 is 0h 5m 3s
total runtime = 0h 17m 31s


In [5]:
#υπολογισμός ακτίνας καμπυλότητας, κρίσιμης συχνότητας και απωλειών

ss = reload(ss)
for w in range(len(init_pos)):
    
    if not (w == 7):
        a = np.load('{}/cyl_{}_pos.npz'.format(dir_name, int(100*init_pos[w])))
        r, ur, phi, uphi, z_cyl, uz_cyl = a['r'], a['ur'], a['phi'], a['uphi'], a['z_cyl'], a['uz_cyl']

        r_curv, nu_crit, p_rad = ss.rc_nc_pr(r, phi, z_cyl, ur, uphi, uz_cyl, Rlc, Delta, delta, gamma0, B_0)

        with open('{}/rc_nc_pr_{}_pos.npz'.format(dir_name, int(100*init_pos[w])), 'wb') as f2:
            np.savez(f2, r_curv = r_curv, nu_crit = nu_crit,  p_rad = p_rad)

        en, ph_num, ph_num_out, ph_en, ph_en_out, ph_num_tot, ph_en_tot = ss.spectrum(r, nu_crit, p_rad, gamma0, Rlc, t)

        with open('{}/spectrum_{}_pos.npz'.format(dir_name, int(100*init_pos[w])), 'wb') as f3:
            np.savez(f3, en = en, ph_num = ph_num, ph_num_out = ph_num_out, ph_en = ph_en, ph_en_out = ph_en_out,
                     ph_num_tot = ph_num_tot, ph_en_tot = ph_en_tot)